In [1]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'E5FEBF04C14581CF182ABE8107AB8B50'
project = Project(api_url, api_key)

#REDCap API configuration for Sensitization
sen_api_url = 'https://redcap.mrc.gm:8443/redcap/api/'
sen_api_key = '9D6259C7B0E732D2C11EAAAF1D49B818'
sen_project = Project(sen_api_url,sen_api_key)


### sensitization

In [2]:
In_sen_data=sen_project.export_records(forms=['indigo_sensitisation'])
sen_log_data=pd.DataFrame(In_sen_data)

sen_log_data=sen_log_data[['in_kiang_number','in_name','in_age','in_village', 'in_compound','in_eden_number','in_contact1','in_contact2','in_contact3','in_contact4','in_outcome','in_study_number']]
sen_log_data=pd.DataFrame(In_sen_data)

sen_log_data = sen_log_data[
    (sen_log_data['in_outcome'] == '1') &
    (sen_log_data['in_kiang_number'].notnull()) &
    (sen_log_data['in_study_number'].notnull())
]



In [3]:
sen_log_data=sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
village_mapping = {
'001':'Dumbuto',
'002':'Sankandi',
'003':'Nioro Jattaba',
'004':'Jattaba',
'005':'Jiffarong',
'006':'Bajana',
'007':'Kuli Kunda',
'008':'Jamaru',
'009':'Brikamanding',
'010':'Kantong Kunda',
'011':'Jali',
'013':'Manduar',
'014':'Bang Kuling',
'015':'Gissay',
'016':'Tankular',
'017':'Joli',
'018':'Kuyang',
'019':'Bantasu',
'020':'Santamba',
'021':'Missira',
'022':'Taborangkoto',
'023':'Burong',
'024':'Jula Kunda',
'025':'Karantaba',
'026':'Mandina',
'027':'Janneh Kunda',
'028':'Kemoto',
'029':'Keneba',
'030':'Batelling',
'031':'Sandeng',
'032':'Wudeba',
'034':'Kenokoto',
'035':'Manari',
'036':'Nineteen',
'040':'WUROKANG',
'041':'KWINELLA SANSANKONO',
'042':'KWINELLA NIA KUNDA',
'043':'TENDABA',
'044':'BUMARR',
'045':'BAMBAKO',
'046':'KUNDONG MARIAYA',
'047':'NEMA',
'048':'KUNDANG NUMU KUNDA',
'049':'KUNDANG FULA KUNDA',
'050':'NEMA KUTA',
'051':'JIRROFF',
'052':'MADINA ANGALLEH',
'053':'JATTA KUNDA',
'054':'MANDINA CENTRAL',
'055':'SARE SARJO',
'056':'SIBETO',
'057':'SARE NDALLA',
'058':'TABANANI',
'060':'WILLINGARA',
'061':'SARE MAMUDU'
    # Add more mappings as needed
}

# Assuming sen_log_data is your DataFrame
sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

## WITHDRAWALS

In [4]:
#extract study termination data
termination_data = project.export_records(forms=['study_termination'])

#convert data to Dataframe.
term_data=pd.DataFrame(termination_data)

In [5]:
# Specify the fields you want to extract base on logics
df_withdrw=term_data[['con_participantid_q1','st_date','st_reason','redcap_event_name']]
df_withdrw=pd.DataFrame(df_withdrw)

#st_date as a date data type
df_withdrw['st_date'] = pd.to_datetime(df_withdrw['st_date'], errors='coerce')


df_withdrw = df_withdrw[
    (df_withdrw['redcap_event_name'] == 'end_of_study_arm_1') &
    (df_withdrw['st_date'].notnull())
]


In [6]:
Withdrawal_df = pd.DataFrame({
    'con_participantid_q1': df_withdrw['con_participantid_q1'],
    'withdrawal date': pd.to_datetime(df_withdrw['st_date']),
    'Reason': df_withdrw['st_reason'],
    'Status': 'Terminated'
})

# Save the DataFrame to a CSV file
Withdrawal_df.to_csv('Withdrawals.csv', index=False)


USS SCAN DATA

In [7]:
##USS SCAN DATA SET
#extract week 20 completed data
USS = project.export_records(forms=['ultrasound_scan'])

#convert data to Dataframe.
uss=pd.DataFrame(USS)

# Specify the fields you want to extract base on logics for week 20
df_uss=uss[['con_participantid_q1','ultsc_date','ultsc_type_q5','redcap_event_name']]
df_uss=pd.DataFrame(df_uss)


In [8]:
# ##EDD LIST
# con_uss = project.export_records(forms=['consent','ultrasound_scan'])
# con_uss=pd.DataFrame(con_uss)

# # Specify the fields you want to extract
# df_data_uss=con_uss[['con_participantid_q1', 'con_participant_eden_num_q11', 'ultsc_date', 'ga_weeks','ga_days','con_cluster_q12','redcap_event_name']]

# df_data_uss = df_data_uss[
#     (df_data_uss['ultsc_date'].notnull())
# ]
# ######################################################################################################################################
# ## EXCLUDE ALL WITHDRAWALS FROM THE DATA
# #Read from the ones that complete withdrawals: "Withdrawals.csv"
# df_withdrawals = pd.read_csv('Withdrawals.csv')

# # Merge the dataframes on 'participant_id'
# con_merged_df = pd.merge(df_data_uss, df_withdrawals, on= 'con_participantid_q1', how='inner')

# # Filter out the common participant IDs
# df_data_uss= df_data_uss[~df_data_uss['con_participantid_q1'].isin(con_merged_df['con_participantid_q1'])]
# ######################################################################################################################################

# # Convert 'ga_weeks' and 'ga_days' columns to numeric type
# df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
# df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# # Calculate today's date
# today = pd.Timestamp.today()

# # Extract the required columns from the DataFrame
# participant_id = df_data_uss['con_participantid_q1']
# uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
# Ga_weeks = df_data_uss['ga_weeks']
# Ga_days = df_data_uss['ga_days']
# eden_num = df_data_uss['con_participant_eden_num_q11']

# # Calculate the total gestational age in days
# total_ga_days = (Ga_weeks * 7) + Ga_days

# # Calculate the estimated due date (EDD)
# edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# # Calculate the current gestational age
# days_diff = (today - uss_date).dt.days
# current_ga = (days_diff + total_ga_days) / 7

# event = df_data_uss['redcap_event_name']
# cluster = df_data_uss['con_cluster_q12']

# ## This data frame is mainted to put together the data and maintain the variable name 'con_participantid_q1' for easy exclusion of participants who completed the wk20.


# edd_df = pd.DataFrame({
#     'con_participantid_q1': participant_id,
#     'USS Date': uss_date,
#     'EDEN | Infant Number': eden_num,
#     'Gestational Age (Weeks)': Ga_weeks,
#     'Gestational Age (Days)': Ga_days,
#     'Cluster': cluster,
#     'Current Gestational Age': current_ga,
#     '(Expected)Delivery Date ((E)DD)': edd,

# })



# edd_df.to_csv("EDD_LIST.CSV")


WK-20 DATA

In [9]:
#ultsc_date as a date data type
df_uss['ultsc_date'] = pd.to_datetime(df_uss['ultsc_date'], errors='coerce')

#extract week 20 USS from the USS scan data set
df_wk20_uss = df_uss[
    (df_uss['redcap_event_name'] == 'week_20_arm_1') &
    (df_uss['ultsc_date'].notnull())
]

# week 20 uss scan data into a data frame
wk20_df = pd.DataFrame({
    'con_participantid_q1' : df_wk20_uss['con_participantid_q1'],
    'uss_date': pd.to_datetime(df_wk20_uss['ultsc_date']),
    'uss_type' : df_wk20_uss['ultsc_type_q5'],
    'Status': 'week 20 completed'
})

# Save the wk20 DataFrame to a CSV file
wk20_df.to_csv('Week 20.csv', index=False)


WK-28 DATA

In [10]:
#extract week 28 USS from the USS scan data set
df_wk28_uss = df_uss[
    (df_uss['redcap_event_name'] == 'week_28_arm_1') &
    (df_uss['ultsc_date'].notnull())
]

# week 28 uss scan data into a data frame
wk28_df = pd.DataFrame({
    'con_participantid_q1' : df_wk28_uss['con_participantid_q1'],
    'uss_date': pd.to_datetime(df_wk28_uss['ultsc_date']),
    'uss_type' : df_wk28_uss['ultsc_type_q5'],
    'Status': 'week 28 completed'
})

# Save the wk28 DataFrame to a CSV file
wk28_df.to_csv('Week 28.csv', index=False)


WK-36 DATA

In [11]:
#extract week 36 USS from the USS scan data set
df_wk36_uss = df_uss[
    (df_uss['redcap_event_name'] == 'week_36_arm_1') &
    (df_uss['ultsc_date'].notnull())
]

# week 36 uss scan data into a data frame
wk36_df = pd.DataFrame({
    'con_participantid_q1' : df_wk36_uss['con_participantid_q1'],
    'uss_date': pd.to_datetime(df_wk36_uss['ultsc_date']),
    'uss_type' : df_wk36_uss['ultsc_type_q5'],
    'Status': 'week 36 completed'
})

# Save the wk36 DataFrame to a CSV file
wk36_df.to_csv('Week 36.csv', index=False)


DELIVERY

In [12]:
#extract delivery data
delivery_data = project.export_records(forms=['delivery'])

#convert data to Dataframe.
delv_data=pd.DataFrame(delivery_data)

In [13]:
#del_date as a date data type
delv_data['del_ddate_q7'] = pd.to_datetime(delv_data['del_ddate_q7'], errors='coerce')

# Specify the fields you want to extract base on logics
df_delv=delv_data[['con_participantid_q1','del_vdate_q1','del_place_q2','del_lbonsetdatetime_q4','del_gestage','del_ddate_q7','redcap_event_name']]
df_delv=pd.DataFrame(df_delv)
df_delv = df_delv[
    (df_delv['redcap_event_name'] == 'delivery__postpart_arm_1') &
    (df_delv['del_ddate_q7'].notnull())
]


In [14]:
#extract required columns

delivery_df = pd.DataFrame({
    'con_participantid_q1': df_delv['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(df_delv['del_ddate_q7']),
    'GA at Delivery': df_delv['del_gestage'],
    'Status': 'Delivered'

})

# Save the DataFrame to a CSV file
delivery_df.to_csv('Delivery.csv', index=False)


INFANT-MOTHER PAIR

In [15]:
#extract infant anthrops data
inf_data = project.export_records(forms=['infant_anthropometry','infant_stool_collection'])
inf_data=pd.DataFrame(inf_data)

inf_data = inf_data[['con_participantid_q1','ina_infantid','ian_date','ian_visitoutcome','infant_anthropometry_complete','isc_dvdate_q1','isc_dtime_q2','isc_stoolcollected_q3','infant_stool_collection_complete','redcap_event_name']]
df_infant = pd.DataFrame(inf_data)

#ian_date as a date data type
df_infant['ian_date'] = pd.to_datetime(df_infant['ian_date'], errors='coerce')
df_infant['isc_dvdate_q1'] = pd.to_datetime(df_infant['isc_dvdate_q1'], errors='coerce')
df_infant['isc_dtime_q2'] = pd.to_datetime(df_infant['isc_dtime_q2'], errors='coerce').dt.time



In [16]:
##Monthly visit conditions
Monthly_visit_conditions =(
    ((df_infant['ian_date'].notnull())) &
    ((df_infant['ian_visitoutcome'].notnull())) &
    (df_infant['infant_anthropometry_complete'] =='2') &
    ((df_infant['isc_dvdate_q1'].notnull())) &
    ((df_infant['isc_dtime_q2'].notnull())) &
    ((df_infant['isc_stoolcollected_q3'].notnull())) &
    (df_infant['infant_stool_collection_complete']== '2')
)

MONTH 1

In [17]:
inf_M1 = df_infant[
    (df_infant['redcap_event_name'] == 'month_1_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m1_infant_mother = df_delv.merge (inf_M1, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M1_infant_mother = m1_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M1_infant_mother = M1_infant_mother[
    ((M1_infant_mother['del_ddate_q7'].notnull())) &
    ((M1_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month1_df = pd.DataFrame({
    'con_participantid_q1': M1_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M1_infant_mother['del_ddate_q7']),
    'Infant ID': M1_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M1_infant_mother['ian_date']),
    'Status': 'Month 1 completed'

})

# Save the DataFrame to a CSV file
Month1_df.to_csv('Month 1.csv', index=False)


Month 2

In [18]:
inf_M2 = df_infant[
    (df_infant['redcap_event_name'] == 'month_2_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m2_infant_mother = df_delv.merge (inf_M2, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M2_infant_mother = m2_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M2_infant_mother = M2_infant_mother[
    ((M2_infant_mother['del_ddate_q7'].notnull())) &
    ((M2_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month2_df = pd.DataFrame({
    'con_participantid_q1': M2_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M2_infant_mother['del_ddate_q7']),
    'Infant ID': M2_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M2_infant_mother['ian_date']),
    'Status': 'Month 2 completed'

})

# Save the DataFrame to a CSV file
Month2_df.to_csv('Month 2.csv', index=False)


Month 3

In [19]:
inf_M3 = df_infant[
    (df_infant['redcap_event_name'] == 'month_3_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m3_infant_mother = df_delv.merge (inf_M3, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M3_infant_mother = m3_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M3_infant_mother = M3_infant_mother[
    ((M3_infant_mother['del_ddate_q7'].notnull())) &
    ((M3_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month3_df = pd.DataFrame({
    'con_participantid_q1': M3_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M3_infant_mother['del_ddate_q7']),
    'Infant ID': M3_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M3_infant_mother['ian_date']),
    'Status': 'Month 3 completed'

})

# Save the DataFrame to a CSV file
Month3_df.to_csv('Month 3.csv', index=False)


Month 4

In [20]:
inf_M4 = df_infant[
    (df_infant['redcap_event_name'] == 'month_4_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m4_infant_mother = df_delv.merge (inf_M4, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M4_infant_mother = m4_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M4_infant_mother = M4_infant_mother[
    ((M4_infant_mother['del_ddate_q7'].notnull())) &
    ((M4_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month4_df = pd.DataFrame({
    'con_participantid_q1': M4_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M4_infant_mother['del_ddate_q7']),
    'Infant ID': M4_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M4_infant_mother['ian_date']),
    'Status': 'Month 4 completed'

})

# Save the DataFrame to a CSV file
Month4_df.to_csv('Month 4.csv', index=False)


Month 5

In [21]:
inf_M5 = df_infant[
    (df_infant['redcap_event_name'] == 'month_5_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m5_infant_mother = df_delv.merge (inf_M5, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M5_infant_mother = m5_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M5_infant_mother = M5_infant_mother[
    ((M5_infant_mother['del_ddate_q7'].notnull())) &
    ((M5_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month5_df = pd.DataFrame({
    'con_participantid_q1': M5_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M5_infant_mother['del_ddate_q7']),
    'Infant ID': M5_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M5_infant_mother['ian_date']),
    'Status': 'Month 5 completed'

})

# Save the DataFrame to a CSV file
Month5_df.to_csv('Month 5.csv', index=False)


Month 6

In [22]:
inf_M6 = df_infant[
    (df_infant['redcap_event_name'] == 'month_6_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m6_infant_mother = df_delv.merge (inf_M6, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M6_infant_mother = m6_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M6_infant_mother = M6_infant_mother[
    ((M6_infant_mother['del_ddate_q7'].notnull())) &
    ((M6_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month6_df = pd.DataFrame({
    'con_participantid_q1': M6_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M6_infant_mother['del_ddate_q7']),
    'Infant ID': M6_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M6_infant_mother['ian_date']),
    'Status': 'Month 6 completed'

})

# Save the DataFrame to a CSV file
Month6_df.to_csv('Month 6.csv', index=False)


Month 7

In [23]:
inf_M7 = df_infant[
    (df_infant['redcap_event_name'] == 'month_7_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m7_infant_mother = df_delv.merge (inf_M7, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M7_infant_mother = m7_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M7_infant_mother = M7_infant_mother[
    ((M7_infant_mother['del_ddate_q7'].notnull())) &
    ((M7_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month7_df = pd.DataFrame({
    'con_participantid_q1': M7_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M7_infant_mother['del_ddate_q7']),
    'Infant ID': M7_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M7_infant_mother['ian_date']),
    'Status': 'Month 7 completed'

})

# Save the DataFrame to a CSV file
Month7_df.to_csv('Month 7.csv', index=False)


Month 8

In [24]:
inf_M8 = df_infant[
    (df_infant['redcap_event_name'] == 'month_8_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m8_infant_mother = df_delv.merge (inf_M8, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M8_infant_mother = m8_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M8_infant_mother = M8_infant_mother[
    ((M8_infant_mother['del_ddate_q7'].notnull())) &
    ((M8_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month8_df = pd.DataFrame({
    'con_participantid_q1': M8_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M8_infant_mother['del_ddate_q7']),
    'Infant ID': M8_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M8_infant_mother['ian_date']),
    'Status': 'Month 8 completed'

})

# Save the DataFrame to a CSV file
Month8_df.to_csv('Month 8.csv', index=False)


Month 9

In [25]:
inf_M9 = df_infant[
    (df_infant['redcap_event_name'] == 'month_9_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m9_infant_mother = df_delv.merge (inf_M9, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M9_infant_mother = m9_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M9_infant_mother = M9_infant_mother[
    ((M9_infant_mother['del_ddate_q7'].notnull())) &
    ((M9_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month9_df = pd.DataFrame({
    'con_participantid_q1': M9_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M9_infant_mother['del_ddate_q7']),
    'Infant ID': M9_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M9_infant_mother['ian_date']),
    'Status': 'Month 9 completed'

})

# Save the DataFrame to a CSV file
Month9_df.to_csv('Month 9.csv', index=False)


Month 10

In [26]:
inf_M10 = df_infant[
    (df_infant['redcap_event_name'] == 'month_10_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m10_infant_mother = df_delv.merge (inf_M10, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M10_infant_mother = m10_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M10_infant_mother = M10_infant_mother[
    ((M10_infant_mother['del_ddate_q7'].notnull())) &
    ((M10_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month10_df = pd.DataFrame({
    'con_participantid_q1': M10_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M10_infant_mother['del_ddate_q7']),
    'Infant ID': M10_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M10_infant_mother['ian_date']),
    'Status': 'Month 10 completed'

})

# Save the DataFrame to a CSV file
Month10_df.to_csv('Month 10.csv', index=False)


Month 11

In [27]:
inf_M11 = df_infant[
    (df_infant['redcap_event_name'] == 'month_11_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m11_infant_mother = df_delv.merge (inf_M11, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M11_infant_mother = m11_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]


M11_infant_mother = M11_infant_mother[
    ((M11_infant_mother['del_ddate_q7'].notnull())) &
    ((M11_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month11_df = pd.DataFrame({
    'con_participantid_q1': M11_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M11_infant_mother['del_ddate_q7']),
    'Infant ID': M11_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M11_infant_mother['ian_date']),
    'Status': 'Month 11 completed'

})

# Save the DataFrame to a CSV file
Month11_df.to_csv('Month 11.csv', index=False)


Month 12

In [28]:
inf_M12 = df_infant[
    (df_infant['redcap_event_name'] == 'month_12_arm_2') &
    Monthly_visit_conditions
]

#merge infant_data to delivery_data base on participant ID
m12_infant_mother = df_delv.merge (inf_M12, on='con_participantid_q1', how='left')

#return the these variables base on the participant Id they have in common
M12_infant_mother = m12_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date']]

M12_infant_mother = M12_infant_mother[
    ((M12_infant_mother['del_ddate_q7'].notnull())) &
    ((M12_infant_mother['ian_date'].notnull()))
]

#extract required columns

Month12_df = pd.DataFrame({
    'con_participantid_q1': M12_infant_mother['con_participantid_q1'],
    'Delivery Date': pd.to_datetime(M12_infant_mother['del_ddate_q7']),
    'Infant ID': M12_infant_mother['ina_infantid'],
    'Infant anthrops Date': pd.to_datetime(M12_infant_mother['ian_date']),
    'Status': 'Month 12 completed'

})

# Save the DataFrame to a CSV file
Month12_df.to_csv('Month 12.csv', index=False)


## CALL LIST

week_20 Call

In [29]:
# Specify the fields you want to extract
con_uss = project.export_records(forms=['consent','ultrasound_scan'])
con_uss=pd.DataFrame(con_uss)

# Specify the fields you want to extract
df_data_uss=con_uss[['con_participantid_q1', 'con_participant_eden_num_q11', 'ultsc_date', 'ga_weeks','ga_days','con_cluster_q12','redcap_event_name']]
df_data_uss=pd.DataFrame(df_data_uss)
df_data_uss = df_data_uss[
    (df_data_uss['redcap_event_name'] == 'recruitment_arm_1') &
    (df_data_uss['ultsc_date'].notnull()) &
    (pd.to_numeric(df_data_uss['ga_weeks']) <= 20)
]


In [30]:
## EXCLUDE ALL WITHDRAWALS FROM THE DATA
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals = pd.read_csv('Withdrawals.csv')

# Merge the dataframes on 'participant_id'
con_merged_df = pd.merge(df_data_uss, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
df_data_uss= df_data_uss[~df_data_uss['con_participantid_q1'].isin(con_merged_df['con_participantid_q1'])]

In [31]:
## EXCLUDE ALL DELIVERIES FROM THE DATA
#Read from the ones that complete withdrawals: "Delivery.csv"
df_delivery = pd.read_csv('Delivery.csv')

# Merge the dataframes on 'participant_id'
del_merged_df = pd.merge(df_data_uss, df_delivery, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
df_data_uss= df_data_uss[~df_data_uss['con_participantid_q1'].isin(del_merged_df['con_participantid_q1'])]

In [32]:
# Convert 'ga_weeks' and 'ga_days' columns to numeric type
df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# Calculate today's date
today = pd.Timestamp.today()

# Extract the required columns from the DataFrame
participant_id = df_data_uss['con_participantid_q1']
uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
Ga_weeks = df_data_uss['ga_weeks']
Ga_days = df_data_uss['ga_days']
eden_num = df_data_uss['con_participant_eden_num_q11']

# Calculate the total gestational age in days
total_ga_days = (Ga_weeks * 7) + Ga_days

# Calculate the estimated due date (EDD)
edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# Calculate the current gestational age
days_diff = (today - uss_date).dt.days
current_ga = (days_diff + total_ga_days) / 7

# Calculate the due dates
due_date = uss_date + pd.to_timedelta(133 - (total_ga_days), unit='days')
last_date = uss_date + pd.to_timedelta(147 - (total_ga_days), unit='days')
visit = 'Week 20'
event = df_data_uss['redcap_event_name']
cluster = df_data_uss['con_cluster_q12']


In [33]:
## This data frame is mainted to put together the data and maintain the variable name 'con_participantid_q1' for easy exclusion of participants who completed the wk20.


wk20_df = pd.DataFrame({
    'con_participantid_q1': participant_id,
    'Cluster': cluster,
    'USS Date': uss_date,
    'EDEN | Infant Number': eden_num,
    'Gestational Age (Weeks)': Ga_weeks,
    'Gestational Age (Days)': Ga_days,
    '(Expected)Delivery Date ((E)DD)': edd,
    'Current Gestational Age': current_ga,
    'Due Date': due_date,
    'Last Date': last_date,
    'Visit': visit
})



In [34]:
## ALL DATA THAT ARE IN THE WEEK 20.CSV EXCLUDED
#Read from the ones that complete week 20: "Week 20.csv"
wk20_uss = pd.read_csv('Week 20.csv')

# Merge the dataframes on 'participant_id'
wk20_merged_df = pd.merge(df_data_uss, wk20_uss, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
wk20_df = wk20_df[~df_data_uss['con_participantid_q1'].isin(wk20_merged_df['con_participantid_q1'])]

wk_28 call

In [35]:
#week 28 data
# Convert 'ga_weeks' and 'ga_days' columns to numeric type
df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# Calculate today's date
today = pd.Timestamp.today()

# Extract the required columns from the DataFrame
participant_id = df_data_uss['con_participantid_q1']
uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
Ga_weeks = df_data_uss['ga_weeks']
Ga_days = df_data_uss['ga_days']
eden_num = df_data_uss['con_participant_eden_num_q11']


# Calculate the total gestational age in days
total_ga_days = (Ga_weeks * 7) + Ga_days

# Calculate the estimated due date (EDD)
edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# Calculate the current gestational age
days_diff = (today - uss_date).dt.days
current_ga = (days_diff + total_ga_days) / 7

# Calculate the due dates
due_date = uss_date + pd.to_timedelta(189 - (total_ga_days), unit='days')
last_date = uss_date + pd.to_timedelta(203 - (total_ga_days), unit='days')
visit = 'Week 28'
event = df_data_uss['redcap_event_name']
cluster = df_data_uss['con_cluster_q12']


In [36]:
wk28_df = pd.DataFrame({
    'con_participantid_q1': participant_id,
    'Cluster': cluster,
    'USS Date': uss_date,
    'EDEN | Infant Number': eden_num,
    'Gestational Age (Weeks)': Ga_weeks,
    'Gestational Age (Days)': Ga_days,
    '(Expected)Delivery Date ((E)DD)': edd,
    'Current Gestational Age': current_ga,
    'Due Date': due_date,
    'Last Date': last_date,
    'Visit': visit
})

In [37]:
## ALL DATA THAT ARE IN THE WEEK 28.CSV EXCLUDED
#Read from the ones that complete week 28: "Week 28.csv"
wk28_uss = pd.read_csv('Week 28.csv')

# Merge the dataframes on 'participant_id'
wk28_merged_df = pd.merge(wk28_df, wk28_uss, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
wk28_df = wk28_df[~wk28_df['con_participantid_q1'].isin(wk28_merged_df['con_participantid_q1'])]


wk_36 call

In [38]:
#week 36 data

# Convert 'ga_weeks' and 'ga_days' columns to numeric type
df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# Calculate today's date
today = pd.Timestamp.today()

# Extract the required columns from the DataFrame
participant_id = df_data_uss['con_participantid_q1']
uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
Ga_weeks = df_data_uss['ga_weeks']
Ga_days = df_data_uss['ga_days']
eden_num = df_data_uss['con_participant_eden_num_q11']


# Calculate the total gestational age in days
total_ga_days = (Ga_weeks * 7) + Ga_days

# Calculate the estimated due date (EDD)
edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# Calculate the current gestational age
days_diff = (today - uss_date).dt.days
current_ga = (days_diff + total_ga_days) / 7

# Calculate the due dates
due_date = uss_date + pd.to_timedelta(245 - (total_ga_days), unit='days')
last_date = uss_date + pd.to_timedelta(259 - (total_ga_days), unit='days')
visit = 'Week 36'
event = df_data_uss['redcap_event_name']
cluster = df_data_uss['con_cluster_q12']


In [39]:
wk36_df = pd.DataFrame({
     'con_participantid_q1': participant_id,
    'Cluster': cluster,
    'USS Date': uss_date,
    'EDEN | Infant Number': eden_num,
    'Gestational Age (Weeks)': Ga_weeks,
    'Gestational Age (Days)': Ga_days,
    '(Expected)Delivery Date ((E)DD)': edd,
    'Current Gestational Age': current_ga,
    'Due Date': due_date,
    'Last Date': last_date,
    'Visit': visit
    
})

In [40]:
## ALL DATA THAT ARE IN THE WEEK 36.CSV EXCLUDED
#Read from the ones that complete week 36: "Week 36.csv"
df_wk36_uss = pd.read_csv('Week 36.csv')

# Merge the dataframes on 'participant_id'
wk36_merged_df = pd.merge(wk36_df, df_wk36_uss, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
wk36_df = wk36_df[~wk36_df['con_participantid_q1'].isin(wk36_merged_df['con_participantid_q1'])]


### POST PARTUM CALL

Infant-mother pair dataframe

In [41]:

#extract neonate screening and infant ID
Neonate = project.export_records(forms=['infantid','neonate_screening'])
Neonate=pd.DataFrame(Neonate)


neonate_data = Neonate[['con_participantid_q1','infantid','neo_vdate' ,'redcap_event_name']]
neonate_data=pd.DataFrame(neonate_data)
neonate_data = neonate_data[
    (neonate_data['redcap_event_name'] == 'baby_check_arm_2') &
    ((neonate_data['neo_vdate'].notnull()))
]


#merge infant_data to delivery_data base on participant ID
neonate_mother = df_delv.merge (neonate_data, on='con_participantid_q1', how='inner')


#return the these variables base on the participant Id they have in common
neonate_mother = neonate_mother[['con_participantid_q1','del_ddate_q7','infantid']]


In [42]:
##Exclude all withdrawals from the delivery data.
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals = pd.read_csv('Withdrawals.csv')

# Merge the dataframes on 'participant_id'
delv_merged_df = pd.merge(neonate_mother, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
neonate_mother= neonate_mother[~neonate_mother['con_participantid_q1'].isin(delv_merged_df['con_participantid_q1'])]

Month 1 call

In [43]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 1)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 1, days=7)

#month 1 data frame
month1_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 1'
})


In [44]:
## ALL DATA THAT ARE IN THE WEEK MONTH 1.csv EXCLUDED
#Read from the ones that complete month 1: "Month 1.csv"
month1 = pd.read_csv('Month 1.csv')

# Merge the dataframes on 'participant_id'
m1_merged_df = pd.merge(month1_df, month1, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month1_df = month1_df[~month1_df['con_participantid_q1'].isin(m1_merged_df['con_participantid_q1'])]



Month 2 call

In [45]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 2)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 2, days=7)

#month 1 data frame
month2_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 2'
})


In [46]:
## ALL DATA THAT ARE IN THE WEEK MONTH 2.csv EXCLUDED
#Read from the ones that complete month 2: "Month 2.csv"
month2 = pd.read_csv('Month 2.csv')

# Merge the dataframes on 'participant_id'
m2_merged_df = pd.merge(month2_df, month2, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month2_df = month2_df[~month2_df['con_participantid_q1'].isin(m2_merged_df['con_participantid_q1'])]



Month 3 call

In [47]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 3)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months= 3, days=7)

#month 1 data frame
month3_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 3'
})


In [48]:
## ALL DATA THAT ARE IN THE WEEK MONTH 3.csv EXCLUDED
#Read from the ones that complete month 3: "Month 3.csv"
month3 = pd.read_csv('Month 3.csv')

# Merge the dataframes on 'participant_id'
m3_merged_df = pd.merge(month3_df, month3, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month3_df = month3_df[~month3_df['con_participantid_q1'].isin(m3_merged_df['con_participantid_q1'])]



Month 4 call

In [49]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=4)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=4, days=7)

#month 4 data frame
month4_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 4'
})

In [50]:
## ALL DATA THAT ARE IN THE WEEK MONTH 4.csv EXCLUDED
#Read from the ones that complete month 4: "Month 3.csv"
month4 = pd.read_csv('Month 4.csv')

# Merge the dataframes on 'participant_id'
m4_merged_df = pd.merge(month4_df, month4, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month4_df = month4_df[~month4_df['con_participantid_q1'].isin(m4_merged_df['con_participantid_q1'])]



Month 5 call

In [51]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=5)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=5, days=7)

#month 5 data frame
month5_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 5'
})

In [52]:
## ALL DATA THAT ARE IN THE WEEK MONTH 5.csv EXCLUDED
#Read from the ones that complete month 5: "Month 5.csv"
month5 = pd.read_csv('Month 5.csv')

# Merge the dataframes on 'participant_id'
m5_merged_df = pd.merge(month5_df, month5, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month5_df = month5_df[~month5_df['con_participantid_q1'].isin(m5_merged_df['con_participantid_q1'])]



Month 6 call

In [53]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=6)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=6, days=7)

#month 6 data frame
month6_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 6'
})

In [54]:
## ALL DATA THAT ARE IN THE WEEK MONTH 6.csv EXCLUDED
#Read from the ones that complete month 6: "Month 6.csv"
month6 = pd.read_csv('Month 6.csv')

# Merge the dataframes on 'participant_id'
m6_merged_df = pd.merge(month6_df, month6, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month6_df = month6_df[~month6_df['con_participantid_q1'].isin(m6_merged_df['con_participantid_q1'])]



Month 7 call

In [55]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=7)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=7, days=7)

#month 7 data frame
month7_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 7'
})

In [56]:
## ALL DATA THAT ARE IN THE WEEK MONTH 7.csv EXCLUDED
#Read from the ones that complete month 7: "Month 7.csv"
month7 = pd.read_csv('Month 7.csv')

# Merge the dataframes on 'participant_id'
m7_merged_df = pd.merge(month7_df, month7, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month7_df = month7_df[~month7_df['con_participantid_q1'].isin(m7_merged_df['con_participantid_q1'])]



Month 8 call

In [57]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=8)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=8, days=7)

#month 8 data frame
month8_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 8'
})

In [58]:
## ALL DATA THAT ARE IN THE WEEK MONTH 8.csv EXCLUDED
#Read from the ones that complete month 8: "Month 8.csv"
month8 = pd.read_csv('Month 8.csv')

# Merge the dataframes on 'participant_id'
m8_merged_df = pd.merge(month8_df, month8, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month8_df = month8_df[~month8_df['con_participantid_q1'].isin(m8_merged_df['con_participantid_q1'])]



Month 9 call

In [59]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=9)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=9, days=7)

#month 9 data frame
month9_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 9'
})

In [60]:
## ALL DATA THAT ARE IN THE WEEK MONTH 9.csv EXCLUDED
#Read from the ones that complete month 9: "Month 9.csv"
month9 = pd.read_csv('Month 9.csv')

# Merge the dataframes on 'participant_id'
m9_merged_df = pd.merge(month9_df, month9, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month9_df = month9_df[~month9_df['con_participantid_q1'].isin(m9_merged_df['con_participantid_q1'])]



Month 10 call

In [61]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=10)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=10, days=7)

#month 10 data frame
month10_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 10'
})

In [62]:
## ALL DATA THAT ARE IN THE WEEK MONTH 10.csv EXCLUDED
#Read from the ones that complete month 10: "Month 10.csv"
month10 = pd.read_csv('Month 10.csv')

# Merge the dataframes on 'participant_id'
m10_merged_df = pd.merge(month10_df, month10, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month10_df = month10_df[~month10_df['con_participantid_q1'].isin(m10_merged_df['con_participantid_q1'])]



Month 11 call

In [63]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=11)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=11, days=7)

#month 11 data frame
month11_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 11'
})

In [64]:
## ALL DATA THAT ARE IN THE WEEK MONTH 11.csv EXCLUDED
#Read from the ones that complete month 11: "Month 11.csv"
month11 = pd.read_csv('Month 11.csv')

# Merge the dataframes on 'participant_id'
m11_merged_df = pd.merge(month11_df, month11, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month11_df = month11_df[~month11_df['con_participantid_q1'].isin(m11_merged_df['con_participantid_q1'])]



Month 12 call

In [65]:
neonate_mother = pd.DataFrame(neonate_mother)

#convert del_ddate_q7 to datetime
neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])

#Due date and last due date calculation
neonate_mother['due_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=12)
neonate_mother['last_date'] = neonate_mother['del_ddate_q7'] + pd.DateOffset(months=12, days=7)

#month 12 data frame
month12_df = pd.DataFrame({
    'con_participantid_q1': neonate_mother['con_participantid_q1'],
    '(Expected)Delivery Date ((E)DD)': pd.to_datetime(neonate_mother['del_ddate_q7']),
    'EDEN | Infant Number':  neonate_mother['infantid'],
    'Due Date': neonate_mother['due_date'],
    'Last Date': neonate_mother['last_date'],
    'Visit': 'Month 12'
})

In [66]:
## ALL DATA THAT ARE IN THE WEEK MONTH 12.csv EXCLUDED
#Read from the ones that complete month 12: "Month 12.csv"
month12 = pd.read_csv('Month 12.csv')

# Merge the dataframes on 'participant_id'
m12_merged_df = pd.merge(month12_df, month12, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
month12_df = month12_df[~month12_df['con_participantid_q1'].isin(m12_merged_df['con_participantid_q1'])]



### FINAL COMPLETE CALL LIST

In [67]:
# This holds all the due dates
df_dueDates1=pd.concat([wk20_df, wk28_df,wk36_df,month1_df,month2_df,month3_df,month4_df,month5_df,month6_df,month7_df,month8_df,month9_df,month10_df,month11_df,month12_df], ignore_index= True)

In [68]:
df_dueDates=pd.merge(df_dueDates1,sen_log_data,on='con_participantid_q1',how='left')

In [69]:
#due date + sensitisation dataframe
df_dueDates = pd.DataFrame({
    'PARTICIPANT ID': df_dueDates['con_participantid_q1'],
    'EDEN | INFANT ID':  df_dueDates['EDEN | Infant Number'],
    'KIANG NUMBER':  df_dueDates['in_kiang_number'],
    'NAME':  df_dueDates['in_name'],
    'CURRENT GA': df_dueDates['Current Gestational Age'],
    'VILLAGE': df_dueDates['in_village'],
    'COMPOUND': df_dueDates['in_compound'],
    'CONTACT1': df_dueDates['in_contact1'],
    'CONTACT2': df_dueDates['in_contact2'],
    'CONTACT3': df_dueDates['in_contact3'],
    'CONTACT4': df_dueDates['in_contact4'],
    'DUE DATE': df_dueDates['Due Date'],
    'LAST DUE DATE': df_dueDates['Last Date'],
    'Visit': df_dueDates['Visit']
})

In [70]:

# Get the current date
current_date = datetime.date.today()

# Find the next Monday
next_monday = today + datetime.timedelta(days=(0 - current_date.weekday() + 7) % 7)

# Convert target_due_date to a datetime object
next_monday = pd.to_datetime(next_monday)

# Calculate the target due date as the next Thursday
target_due_date = next_monday + datetime.timedelta(days=3)

# Convert target_due_date to a datetime object
target_due_date = pd.to_datetime(target_due_date)

df_callList = df_dueDates[df_dueDates['DUE DATE'] <= target_due_date]


In [71]:
# Get today's date and time
now = datetime.datetime.now()

# Format the current date and time
current_datetime = now.strftime('%Y%m%d_%H%M%S')

# Create the filename with the current date and time
filename = f'CallList_{current_datetime}.csv'

 #Save the filtered DataFrame to a CSV file
df_callList.to_csv(filename, index=False)